Pewien słuchacz szkoły muzycznej ma w sobie niesamowity talent. Jednak przed jej ukończeniem wstrzymuje go jeden przedmiot - "Kompozytorzy muzyki klasycznej". Słuchacz ten, mając dość niepowodzeń w zdawaniu tego tematu, zwraca się do Was o pomoc.

Zadanie polega na stworzeniu modelu rekurencyjnego, który będzie przewidywał kompozytora danego utworu klasycznego w oparciu o jego zapis w formie sekwencji akordów. Akordy znormalizowane zostały do klucza C-dur lub a-moll, w zależności od skali utworu (durowa/molowa).
Dane przygotowane są w postaci pickle (https://docs.python.org/3/library/pickle.html), w których znajduje się lista krotek z sekwencjami i odpowiadającymi im klasami (kompozytorami), odpowiednio: {0: 'bach', 1: 'beethoven', 2: 'debussy', 3: 'scarlatti', 4: 'victoria'}. Dane treningowe znajdują się w pliku train.pkl. W pliku test_no_target.pkl znajdują się testowe sekwencje, dla których predykcje mają Państwo przewidzieć.

Uwaga, utwory mogą mieć różne długości. Do stworzenia batchy dla przykładów różnej długości proszę wykorzystać omówiony na zajęciach padding i trenować z wykorzystaniem wyrównanych tensorów lub spakowanych sekwencji (PackedSequence).

Bardzo proszę, żeby zwrócili Państwo archiwum zip, zgodnie z instrukcjami:
- Archiwum powinno być nazwane {poniedzialek/piatek}_nazwisko1_nazwisko2.zip, bez nawiasów klamrowych przy dniu tygodnia
- W archiwum proszę, bez zbędnych podfolderów, umieścić pliki ze swoim kodem i testowe predykcje nazwane {poniedzialek/piatek}_nazwisko1_nazwisko2.csv (lub nazwa drużyny), bez nawiasów klamrowych przy dniu tygodnia
- Testowe predykcje powinny mieć kolejność zgodną z kolejnością sekwencji w picklu. Plik .csv nie powinien mieć nagłówka ani indeksów.

Proszę zwracać uwagę na prawidłowe nazewnictwo oraz odpowiedni format zwracanych plików. Niedostosowanie się do wytycznych może spowodować nieuwzględnienie Państwa w rankingu i utratę punktów za osiągnięty wynik!
Proszę także o udokumentowanie wykonanych eksperymentów.

In [105]:
import pickle
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from torch.functional import F
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [106]:
DATA_PATH = './p5'
MODEL_PATH = './model'

BATCH_SIZE = 32
LEARNING_RATE = 0.001
TEST_RATIO = 0.2
EPOCHS = 101

MAX_DATASET_LEN = 750

HIDDEN_DIM = 128
INPUT_DIM = 1
OUTPUT_DIM = 5
DROPOUT_RATE = 0.5

In [107]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

Device: cuda


In [108]:
with open(f'{DATA_PATH}/train.pkl', 'rb') as file:
    train_data = pickle.load(file)

with open(f'{DATA_PATH}/test_no_target.pkl', 'rb') as file:
    test_data = pickle.load(file)

In [109]:
class MusicDataset(Dataset):
    def __init__(self, data, max_len=None, is_train_dataset=True):
        self.seq, self.labels = zip(*data)
        self.max_len = max(len(s) for s in self.seq)
        self.seq = [MusicDataset.pad_collate(self, seq) for seq in self.seq]
        self.is_train_dataset = is_train_dataset
    
    def __len__(self):
        return len(self.seq)
    
    def __getitem__(self, idx):
        return self.seq[idx], self.labels[idx] if self.is_train_dataset else self.seq[idx]

    @staticmethod
    def pad_collate(self, seq, pad_value=0):
        seq = torch.tensor(seq, dtype=torch.float32).unsqueeze(1)
        padding = self.max_len - len(seq)
        return F.pad(seq, (0, 0, 0, padding), mode="constant", value=pad_value)


In [110]:
train_data, validation_data = train_test_split(train_data, test_size=TEST_RATIO)

train_dataset = MusicDataset(train_data, max_len=MAX_DATASET_LEN, is_train_dataset=True)
test_dataset = MusicDataset([(seq, None) for seq in test_data], max_len=MAX_DATASET_LEN, is_train_dataset=False)
validation_dataset = MusicDataset(validation_data, max_len=MAX_DATASET_LEN, is_train_dataset=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [111]:
class ComposerClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers=1):
        super(ComposerClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, 
                            hidden_dim, 
                            num_layers=n_layers, 
                            bidirectional=True, 
                            batch_first=True,
                            dropout=DROPOUT_RATE)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(DROPOUT_RATE)
        self.batch_norm = nn.BatchNorm1d(hidden_dim * 2)
    
    def forward(self, x):
        lstm_out, (hidden, cell) = self.lstm(x)
        x = torch.cat((hidden[-2], hidden[-1]), dim=1)
        x = self.batch_norm(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [112]:
_, train_labels = zip(*train_data)
class_nums = Counter(train_labels)
samples_num = sum(class_nums.values())
class_weights = {cls: samples_num / num for cls, num in class_nums.items()}
weights = torch.tensor([class_weights[ii] for ii in range(len(class_nums))], dtype=torch.float32).to(device)

In [113]:
model = ComposerClassifier(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [114]:
best_validation_loss = float('inf')
best_validation_accuracy = 0.0
early_stop_num = 0
early_stop_threshold = 5
train_loss = []
train_accuracy = []
validation_loss = []
validation_accuracy = []

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    
    for sequences, labels in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

        outputs = outputs.cpu()
        labels = labels.cpu()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_accuracy.append(correct / total)
    train_loss.append(total_loss / len(train_loader))

    print(f'Epoch {epoch + 1}/{EPOCHS} - Train Loss: {train_loss[-1]:.4f} - Train Accuracy: {train_accuracy[-1]:.4f}')

    model.eval()
    validation_loss_epoch = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for sequences, labels in validation_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)

            outputs = model(sequences)
            loss = criterion(outputs, labels)

            validation_loss_epoch += loss.item()

            outputs = outputs.cpu()
            labels = labels.cpu()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    validation_accuracy_epoch = correct / total
    validation_loss.append(validation_loss_epoch / len(validation_loader))
    average_validation_loss = validation_loss_epoch / len(validation_loader)

    print(f'Epoch {epoch + 1}/{EPOCHS} - Validation Loss: {average_validation_loss:.4f} - Validation Accuracy: {validation_accuracy_epoch:.4f}')
    
    if validation_accuracy_epoch > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy_epoch
        torch.save(model.state_dict(), f'{MODEL_PATH}/best.pth')
    
print('Best accuracy:', best_validation_accuracy)

Epoch 1/101 - Train Loss: 1.4220 - Train Accuracy: 0.4134
Epoch 1/101 - Validation Loss: 1.1297 - Validation Accuracy: 0.5901


KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load(f'{MODEL_PATH}/best.pth'))

predictions = []

with torch.no_grad():
    model.eval()
    
    for seq in test_loader:
        seq = seq.to(device)
        outputs = model(seq)
        outputs = outputs.cpu()
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

df = pd.DataFrame(predictions)
df.to_csv('poniedzialek_kieruczenko_ziarek.csv', header=False, index=False)

In [ ]:
epochs = range(1, EPOCHS + 1)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, validation_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_accuracy, label='Training Accuracy')
plt.plot(epochs, validation_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()